In [3]:
from loss_functions import DiceLoss
from train_test import train_and_test
from dataset_setup import train_test_split, CustomDataset, augment_image, DataLoader

from models.resnet import resnet50

import os
import torch


In [ ]:
image_folder = f'../train/kidney_2/images/'
image_files = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(".tif")]
image_files.sort()
labels_folder = f'../train/kidney_2/labels/'
label_files = [os.path.join(labels_folder, img) for img in os.listdir(labels_folder) if img.endswith(".tif")]
label_files.sort()

In [ ]:
train_image_files, val_image_files, train_mask_files, val_mask_files = train_test_split(
    image_files, label_files, test_size=0.1, random_state=42)

train_dataset = CustomDataset(train_image_files, train_mask_files, augmentation_transforms=augment_image, input_size=(256, 256))
val_dataset = CustomDataset(val_image_files, val_mask_files, augmentation_transforms=augment_image, input_size=(256, 256))

train_dataloader= DataLoader(train_dataset, batch_size=28, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=28, shuffle=False)

dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader
}

In [ ]:
epochs = 5
def train():
    model = resnet50()
    model.load_state_dict(torch.load('pretrain/resnet_50_23dataset.pth'))  
    optimizer = torch.optim.Adam(model.parameters())
    criterion = DiceLoss()
    trained_model, train_epoch_losses, test_epoch_losses = train_and_test(model, dataloaders, optimizer, criterion, num_epochs=epochs, show_images=True)
    return trained_model, train_epoch_losses, test_epoch_losses


trained_model, train_epoch_losses, test_epoch_losses = train()
#torch.save(trained_model.state_dict(), 'lower_learning_rate_100.pth')